This notebook implements experiments on standard (unfair) settings with 2D Rectangles

In [1]:
import sys
sys.path.append('../../..')

import numpy as np
import random
import pandas as pd
import time
from matplotlib import pyplot as plt

from core.points import *
from core.ranges import *
from algorithms.epsnet import *
from core.ranges import get_range_space
from core.verification import is_epsnet

Sampling methods:

In [6]:
def report_epsnet_sample(n, m, eps, c1=1):
    # n = 2**10
    # m = 2**12
    # eps = 0.4
    success_prob = 0.9

    points = [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=0) for _ in range(n // 2)]
    points += [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=1) for _ in range(n // 2)]
    ranges = []
    for _ in range(m):
        xmin = random.uniform(0, 1)
        ymin = random.uniform(0, 1)
        xmax = random.uniform(xmin, 1)  # Ensure xmax > xmin
        ymax = random.uniform(ymin, 1)  # Ensure ymax > ymin
        ranges.append(RectangleRange(xmin=xmin, ymin=ymin, xmax=xmax, ymax=ymax))
    rangespace = get_range_space(points, ranges)
    vc = 2

    heavy_ranges = []
    for r in rangespace:
        if len(r) >= n * eps:
            heavy_ranges.append(r)
            
    print(f"n: {n}, m: {m}, vc: {vc}, epsilon: {eps}, number of heavy ranges: {len(heavy_ranges)}")

    # for _ in range(10):
    start = time.time()
    epsnet = build_epsnet_sample(
        points=points,
        rangespace=rangespace,
        vc=vc,
        epsilon=eps,
        success_prob=success_prob,
        weights=None,
        c1=c1
    )
    end = time.time()

    print(f"Number of points in epsnet: {len(epsnet)}, time taken: {end - start:.6f} seconds")

    success = is_epsnet(epsnet=epsnet, rangespace=rangespace, epsilon=eps)
    print(f"Success: {success}")  # Verify the eps-net
    
    return (n, m, 
            success_prob, eps, end - start, success, 
            len(heavy_ranges), 
            len([p for p in epsnet if p.color == 0]), len([p for p in epsnet if p.color == 1]))

In [7]:
n_values = [2**9, 2**10, 2**11, 2**12]
m_values = [2**9, 2**10, 2**11, 2**12]
eps_values = [0.2, 0.3, 0.4, 0.5, 0.6]

aggregated_results = {
    "n": [],
    "m": [],
    "prob": [],
    "eps": [],
    "time": [],
    "success": [],
    "heavy_ranges": [],
    "tries": [],
    "red_points": [],
    "blue_points": []
}

for n in n_values:
    for m in m_values:
        for eps in eps_values:
            tries = 10
            print(f"Running for n={n}, m={m}, eps={eps}")
            result = report_epsnet_sample(n, m, eps)
            while not result[5] and tries > 0:
                print("Retrying...")
                result = report_epsnet_sample(n, m, eps)
                tries -= 1
            
            aggregated_results["n"].append(result[0])
            aggregated_results["m"].append(result[1])
            aggregated_results["prob"].append(result[2])
            aggregated_results["eps"].append(result[3])
            aggregated_results["time"].append(result[4])
            aggregated_results["success"].append(result[5])
            aggregated_results["heavy_ranges"].append(result[6])
            aggregated_results["tries"].append(10 - tries)
            aggregated_results["red_points"].append(result[7])
            aggregated_results["blue_points"].append(result[8])

result = pd.DataFrame(aggregated_results)

Running for n=512, m=512, eps=0.2
n: 512, m: 512, vc: 2, epsilon: 0.2, number of heavy ranges: 33
[build_epsnet_sample] epsnet size m: 506
Number of points in epsnet: 506, time taken: 0.000076 seconds
Success: True
Running for n=512, m=512, eps=0.3
n: 512, m: 512, vc: 2, epsilon: 0.3, number of heavy ranges: 14
[build_epsnet_sample] epsnet size m: 306
Number of points in epsnet: 306, time taken: 0.000044 seconds
Success: True
Running for n=512, m=512, eps=0.4
n: 512, m: 512, vc: 2, epsilon: 0.4, number of heavy ranges: 3
[build_epsnet_sample] epsnet size m: 213
Number of points in epsnet: 213, time taken: 0.000033 seconds
Success: True
Running for n=512, m=512, eps=0.5
n: 512, m: 512, vc: 2, epsilon: 0.5, number of heavy ranges: 2
[build_epsnet_sample] epsnet size m: 160
Number of points in epsnet: 160, time taken: 0.000027 seconds
Success: True
Running for n=512, m=512, eps=0.6
n: 512, m: 512, vc: 2, epsilon: 0.6, number of heavy ranges: 1
[build_epsnet_sample] epsnet size m: 127
Numb

In [ ]:
# result.to_csv("epsnet_sample_results_rectangle.csv", index=False)

In [ ]:
grouped_result = result.groupby("eps")["time"].mean().reset_index()

# Plot the aggregated data
plt.figure(figsize=(8, 5))
plt.plot(grouped_result["eps"], grouped_result["time"], marker='o')
plt.title("Aggregated Time vs eps")
plt.xlabel("eps")
plt.ylabel("Time (seconds)")
# plt.xscale('log', base=2)
plt.grid()
plt.show()

Discrepancy method:

In [14]:
def report_epsnet_discrepancy(n, m, eps):
    # n = 2**10
    # m = 2**12
    # eps = 0.4
    # success_prob = 0.9

    points = [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=0) for _ in range(n // 2)]
    points += [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=1) for _ in range(n // 2)]
    ranges = []
    for _ in range(m):
        xmin = random.uniform(0, 1)
        ymin = random.uniform(0, 1)
        xmax = random.uniform(xmin, 1)  # Ensure xmax > xmin
        ymax = random.uniform(ymin, 1)  # Ensure ymax > ymin
        ranges.append(RectangleRange(xmin=xmin, ymin=ymin, xmax=xmax, ymax=ymax))
    rangespace = get_range_space(points, ranges)
    vc = 2

    heavy_ranges = []
    for r in rangespace:
        if len(r) >= n * eps:
            heavy_ranges.append(r)
            
    print(f"n: {n}, m: {m}, vc: {vc}, epsilon: {eps}, number of heavy ranges: {len(heavy_ranges)}")

    # for _ in range(10):
    start = time.time()
    epsnet = build_epsnet_discrepancy(
        points=points,
        rangespace=rangespace,
        vc=vc,
        epsilon=eps,
    )
    end = time.time()

    print(f"Number of points in epsnet: {len(epsnet)}, time taken: {end - start:.6f} seconds")

    success = is_epsnet(epsnet=epsnet, rangespace=rangespace, epsilon=eps)
    print(f"Success: {success}")  # Verify the eps-net
    
    return (n, m, 0.9, eps, end - start, success, len(heavy_ranges)
            , len([p for p in epsnet if p.color == 0]), len([p for p in epsnet if p.color == 1]))

In [15]:
n_values = [2**9, 2**10, 2**11, 2**12]
m_values = [2**9, 2**10, 2**11, 2**12]
eps_values = [0.2, 0.3, 0.4, 0.5, 0.6]

aggregated_results = {
    "n": [],
    "m": [],
    "prob": [],
    "eps": [],
    "time": [],
    "success": [],
    "heavy_ranges": [],
    "tries": [],
    "red_points": [],
    "blue_points": []
}

for n in n_values:
    for m in m_values:
        for eps in eps_values:
            print(f"Running for n={n}, m={m}, eps={eps}")
            result = report_epsnet_discrepancy(n, m, eps)
            
            aggregated_results["n"].append(result[0])
            aggregated_results["m"].append(result[1])
            aggregated_results["prob"].append(result[2])
            aggregated_results["eps"].append(result[3])
            aggregated_results["time"].append(result[4])
            aggregated_results["success"].append(result[5])
            aggregated_results["heavy_ranges"].append(result[6])
            aggregated_results["tries"].append(0)
            aggregated_results["red_points"].append(result[7])
            aggregated_results["blue_points"].append(result[8])

result = pd.DataFrame(aggregated_results)

Running for n=512, m=512, eps=0.2
n: 512, m: 512, vc: 2, epsilon: 0.2, number of heavy ranges: 37
[build_epsnet_discrepancy] epsnet size m: 506
Number of points in epsnet: 512, time taken: 0.000017 seconds
Success: True
Running for n=512, m=512, eps=0.3
n: 512, m: 512, vc: 2, epsilon: 0.3, number of heavy ranges: 19
[build_epsnet_discrepancy] epsnet size m: 306
Number of points in epsnet: 512, time taken: 0.000012 seconds
Success: True
Running for n=512, m=512, eps=0.4
n: 512, m: 512, vc: 2, epsilon: 0.4, number of heavy ranges: 4
[build_epsnet_discrepancy] epsnet size m: 213
[_greedy_discrepancy_halving] counter: 256 / 256
Number of points in epsnet: 256, time taken: 0.760329 seconds
Success: True
Running for n=512, m=512, eps=0.5
n: 512, m: 512, vc: 2, epsilon: 0.5, number of heavy ranges: 1
[build_epsnet_discrepancy] epsnet size m: 160
[_greedy_discrepancy_halving] counter: 256 / 256
Number of points in epsnet: 256, time taken: 0.745351 seconds
Success: True
Running for n=512, m=512

In [ ]:
# result.to_csv("epsnet_discrepancy_results_rectangle.csv", index=False)

Sketch and Merge:

In [39]:
def report_epsnet_sketch_merge(n, m, eps):
    # n = 2**10
    # m = 2**12
    # eps = 0.4
    # success_prob = 0.9

    points = [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=0) for _ in range(n // 2)]
    points += [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=1) for _ in range(n // 2)]
    ranges = []
    for _ in range(m):
        xmin = random.uniform(0, 1)
        ymin = random.uniform(0, 1)
        xmax = random.uniform(xmin, 1)  # Ensure xmax > xmin
        ymax = random.uniform(ymin, 1)  # Ensure ymax > ymin
        ranges.append(RectangleRange(xmin=xmin, ymin=ymin, xmax=xmax, ymax=ymax))
    rangespace = get_range_space(points, ranges)
    vc = 2

    heavy_ranges = []
    for r in rangespace:
        if len(r) >= n * eps:
            heavy_ranges.append(r)
            
    print(f"n: {n}, m: {m}, vc: {vc}, epsilon: {eps}, number of heavy ranges: {len(heavy_ranges)}")

    # for _ in range(10):
    start = time.time()
    epsnet = build_epsnet_sketch_merge(
        points=points,
        rangespace=rangespace,
        vc=vc,
        epsilon=eps,
        c1=0
    )
    end = time.time()

    print(f"Number of points in epsnet: {len(epsnet)}, time taken: {end - start:.6f} seconds")

    success = is_epsnet(epsnet=epsnet, rangespace=rangespace, epsilon=eps)
    print(f"Success: {success}")  # Verify the eps-net
    
    return (n, m, 0.9, eps, end - start, success, len(heavy_ranges)
            , len([p for p in epsnet if p.color == 0]), len([p for p in epsnet if p.color == 1]))

In [41]:
n_values = [2**9, 2**10, 2**11]#, 2**12]
m_values = [2**9, 2**10, 2**11, 2**12]
eps_values = [0.2, 0.3, 0.4, 0.5, 0.6]

aggregated_results = {
    "n": [],
    "m": [],
    "prob": [],
    "eps": [],
    "time": [],
    "success": [],
    "heavy_ranges": [],
    "tries": [],
    "red_points": [],
    "blue_points": []
}

for n in n_values:
    for m in m_values:
        for eps in eps_values:
            print(f"Running for n={n}, m={m}, eps={eps}")
            result = report_epsnet_sketch_merge(n, m, eps)
            
            aggregated_results["n"].append(result[0])
            aggregated_results["m"].append(result[1])
            aggregated_results["prob"].append(result[2])
            aggregated_results["eps"].append(result[3])
            aggregated_results["time"].append(result[4])
            aggregated_results["success"].append(result[5])
            aggregated_results["heavy_ranges"].append(result[6])
            aggregated_results["tries"].append(0)
            aggregated_results["red_points"].append(result[7])
            aggregated_results["blue_points"].append(result[8])

result = pd.DataFrame(aggregated_results)

Running for n=512, m=512, eps=0.2
n: 512, m: 512, vc: 2, epsilon: 0.2, number of heavy ranges: 43
[build_epsnet_sketch_merge] epsnet size m: 506
[build_epsnet_sketch_merge] partition size p: 512
[build_epsnet_sketch_merge] Starting sketch-and-merge...

Number of points in epsnet: 512, time taken: 0.000035 seconds
Success: True
Running for n=512, m=512, eps=0.3
n: 512, m: 512, vc: 2, epsilon: 0.3, number of heavy ranges: 16
[build_epsnet_sketch_merge] epsnet size m: 306
[build_epsnet_sketch_merge] partition size p: 512
[build_epsnet_sketch_merge] Starting sketch-and-merge...

Number of points in epsnet: 512, time taken: 0.000026 seconds
Success: True
Running for n=512, m=512, eps=0.4
n: 512, m: 512, vc: 2, epsilon: 0.4, number of heavy ranges: 7
[build_epsnet_sketch_merge] epsnet size m: 213
[build_epsnet_sketch_merge] partition size p: 256
[build_epsnet_sketch_merge] Starting sketch-and-merge...
[_sketch_merge] pair: 1 / 1 of total nodes: 2
[_greedy_discrepancy_halving] counter: 256 / 

In [ ]:
# result.to_csv("epsnet_sketch_merge_results_rectangle.csv", index=False)